In [1]:
import sys

sys.path.append('../chunk_caption_index_component/')
sys.path.append('../enhanced_doc_analyzer_component/')

In [2]:
# Example of registering the component in a workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Get workspace
ml_client = MLClient.from_config(
    credential=DefaultAzureCredential()
)

Found the config file in: /config.json


In [3]:
from azure.ai.ml import load_component

chunk_caption_index = load_component(source="../chunk_caption_index_component/chunk-caption-index-component.yaml")

In [4]:
# ml_client.components.create_or_update(chunk_caption_index)

In [5]:
from azure.ai.ml import dsl

@dsl.pipeline
def document_processing_pipeline(
    input_folder,
    azure_openai_connection_id,
    azure_search_connection_id,
    embd_deployment_name,
    vision_deployment_name,
    index_name
):
    chunk_caption_job = chunk_caption_index(
        input_folder=input_folder,
        azure_openai_connection_id=azure_openai_connection_id,
        azure_search_connection_id=azure_search_connection_id,
        embd_deployment_name=embd_deployment_name,
        vision_deployment_name=vision_deployment_name,
        index_name=index_name
    )
    return {"output_folder": chunk_caption_job.outputs.output_folder}

In [ ]:
doc_intelligence_connection = ml_client.connections.get("my-doc-intelligence-connection")
azure_search_connection = ml_client.connections.get("aisearch505")
azuee_openai_connection = ml_client.connections.get("aoai-sweden-505")

pipeline = document_processing_pipeline(
    input_folder="azureml:azureml_1a4dfa77-6671-4737-b4b3-61df944ea4b3_output_data_output_dir:1",
    azure_openai_connection_id=azuee_openai_connection.id,
    azure_search_connection_id=azure_search_connection.id,
    embd_deployment_name="text-embedding-ada-002",
    vision_deployment_name="gpt-4v",
    index_name="myindex"
)

# Submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline,
    experiment_name="document_processing_pipeline",
    tags={"tag1": "value1", "tag2": "value2"}
)



AttributeError: 'PipelineJob' object has no attribute 'submit'